# 4a Map Matching Routed Paths OpenStreetMap

The goal of the following notebook is to mapmatch the Routed Paths into the SUMO road network.
___

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
import sumolib
from tqdm import tqdm
from mapmatch import sumo_map_matching, tdf_from_edgelist
from utils import create_xml_vehicles
import folium

#### File paths

In [ ]:
# road network path
road_network_path = "../sumo_simulation_data/milan_road_network.net.xml"


# osm routed paths path
osm_routed_paths_path = "../data/dicts_suggestion/dict_results_osm_[0_20].json"


# dict mobility demand path
dict_mobility_demand_path = "../data/dict_mobility_demand.json"


# output routed paths path
output_routed_paths_filename = "../sumo_simulation_data/routed_paths/routed_paths_osm.rou.xml"

####  Load road networks

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)
road_network_int = sumolib.net.readNet(road_network_path, withInternal=True)

#### Load the GPS routed paths

In [ ]:
with open(osm_routed_paths_path, 'r') as f:
    gps_osm_routed_paths = json.load(f)

print(str(len(gps_osm_routed_paths))+" GPS routed paths.")

####  Load mobility demand dictionary

In [ ]:
with open(dict_mobility_demand_path) as json_file:
    dict_md = json.load(json_file)
    
print(str(len(dict_md))+" mobility demands.")

list_id_vehicles = [v_id for v_id in dict_md.keys() if "vehicle" in v_id]

In [ ]:
#specify the range of vehicles for which to apply the map matching

id_from, id_to =  0, len(list_id_vehicles)

print("from id "+str(id_from)+" to id "+str(id_to))

#### Apply the map-matching algorithm

In [ ]:
dict_map_matched = {}

pbar = tqdm(total=len(list_id_vehicles[id_from: id_to]))

for v in list_id_vehicles[id_from: id_to]:
    
    points = gps_osm_routed_paths[v]['points']
    edge_from = dict_md[v]['edges'][0]
    edge_to = dict_md[v]['edges'][1]
        
    best_edges, sspd_best, length, conf = sumo_map_matching(points, road_network, road_network_path, 
                                                            road_network_int, edge_from, edge_to, accept_th=5e-5,
                                                            early_stop=True)

    
    dict_map_matched[v] = {'mapped_edges':best_edges, 'length':length, 'sspd':sspd_best,
                          'best_conf': conf}
    
    pbar.update(1)

In [ ]:
for k in dict_map_matched:
    print(dict_map_matched[k]['sspd']);

### If you want to visually inspect the map matching use the following cell

In [ ]:
v_id = "vehicle_0"

points_traj = gps_osm_routed_paths[v_id]['points']

m = folium.Map(location=[points_traj[0][1], points_traj[0][0]], tiles='cartodbpositron', zoom_start=13)

# OSM path
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in points_traj], weigth=3, color="blue").add_to(m)

# map matched path
tdf_mapmatched = tdf_from_edgelist(dict_map_matched[v_id]['mapped_edges'], road_network)
points_mapmatched = tdf_mapmatched[['lng','lat']].values

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in points_mapmatched], weigth=3, color="orange").add_to(m)

m

### Create a dictionary describing the mapped edges for each vehicle

In [ ]:
dict_sumo_osm = {}

for ind, v_id in enumerate(list(dict_md.keys())):      
    if "vehicle" in v_id:
        new_id = "osm_"+v_id.split("_")[1]
        dep_time = dict_md[v_id]['time']
        edges_v = dict_map_matched[v_id]['mapped_edges']
        
        dict_sumo_osm[new_id] = {'edges':str(edges_v).replace(",","").replace("'","")[1:-1], 'time': dep_time}

### Create a SUMO xml file that described the OSM routed paths

In [ ]:
create_xml_vehicles(dict_sumo_osm, output_routed_paths_filename);